<a href="https://colab.research.google.com/github/rashida048/Apache-Beam/blob/main/additional_inputs_and_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 116.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.9/648.9 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 115.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 27.9 MB/s eta 0:00:00


In [ ]:
mkdir -p data

In [ ]:
ls

data/  sample_data/


In [ ]:
import apache_beam as beam

In [ ]:
#ParDo as Map
class SplitRow(beam.DoFn):
  def process(self, element):
    return [element.split(',')]

p1 = beam.Pipeline()
attendance_count = (p1
                    |beam.io.ReadFromText('data/dept_data.txt')
                    |beam.ParDo(SplitRow())
                    #|beam.Filter(filtering)
                    #|beam.Map(lambda record: (record[1], 1))
                    #|beam.CombinePerKey(sum)
                    |beam.io.WriteToText('data/output_new1'))

p1.run()
!head -n 5 data/output_new1-00000-of-00001

['149633CM', 'Marco', '10', 'Accounts', '1-01-2019']
['212539MU', 'Rebekah', '10', 'Accounts', '1-01-2019']
['231555ZZ', 'Itoe', '10', 'Accounts', '1-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '1-01-2019']
['704275DC', 'Kyle', '10', 'Accounts', '1-01-2019']


In [ ]:
#ParDo as FlatMap
class SplitRow(beam.DoFn):
  def process(self, element):
    return element.split(',')

p2 = beam.Pipeline()
attendance_count = (p2
                    |beam.io.ReadFromText('data/dept_data.txt')
                    |beam.ParDo(SplitRow())
                    #|beam.Filter(filtering)
                    #|beam.Map(lambda record: (record[1], 1))
                    #|beam.CombinePerKey(sum)
                    |beam.io.WriteToText('data/output_new2'))

p2.run()
!head -n 10 data/output_new2-00000-of-00001

149633CM
Marco
10
Accounts
1-01-2019
212539MU
Rebekah
10
Accounts
1-01-2019


In [ ]:
def SplitRow(element):
  return element.split(',')

def filtering(record):
  return record[3] == "Accounts"

p6 = beam.Pipeline()
attendance_count = (p6
                    |beam.io.ReadFromText('data/dept_data.txt')
                    |beam.Map(SplitRow)
                    #|beam.Map(lambda record: record.split(',))
                    |beam.Filter(filtering)
                    #|beam.Map(lambda record: record[3] == 'Account')
                    |beam.Map(lambda record: (record[1], 1))
                    |beam.io.WriteToText('data/output_new6'))

p6.run()
!head -n 5 data/output_new6-00000-of-00001

('Marco', 1)
('Rebekah', 1)
('Itoe', 1)
('Edouard', 1)
('Kyle', 1)


In [ ]:
#ParDo as FlatMap
class SplitRow(beam.DoFn):
  def process(self, element):
    return [element.split(',')]

class FilterAccountsEmployee(beam.DoFn):
  def process(self, element):
    if element[3] == 'Accounts':
        return [element]

class PairEmployees(beam.DoFn):
  def process(self, element):
    return [(element[3]+ ', '+ element[1], 1)]

class Counting(beam.DoFn):
  def process(self, element):
    (key, values) = element
    return [(key, sum(values))]


p4 = beam.Pipeline()
attendance_count = (p4
                    |beam.io.ReadFromText('data/dept_data.txt')
                    |beam.ParDo(SplitRow())
                    |beam.ParDo(FilterAccountsEmployee())
                    |beam.ParDo(PairEmployees())
                    | 'Group' >> beam.GroupByKey()
                    | 'Sum Using ParDo' >> beam.ParDo(Counting())
                    #|beam.Map(lambda record: (record[1], 1))
                    #|beam.CombinePerKey(sum)
                    |beam.io.WriteToText('data/output_new4'))

p4.run()
!head -n 10 data/output_new4-00000-of-00001

('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)


**Combiner**

In [ ]:
class AverageFn(beam.CombineFn):
  def create_accumulator(self):
    return (0.0, 0) #initialize (sum, count)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum+input, count+1

  def merge_accumulators(self, accumulators):
    ind_sums, ind_counts = zip(*accumulators) #zip - [(27, 3), (39, 3), (18, 2)] --> [(27, 39, 18), (3, 3, 2)]
    return sum(ind_sums), sum(ind_counts)   #(84, 8)

  def extract_output(self, sum_count):
    (sum, count) = sum_count
    return sum / count if count else float('NaN')


p5 = beam.Pipeline()

small_sum = (p5
             |beam.Create([15, 5, 7, 7, 9, 23, 13, 5])
             |beam.CombineGlobally(AverageFn())
             | 'Write results' >> beam.io.WriteToText('data/combine')
)

p5.run()

In [ ]:
!head -n 10 data/combine-00000-of-00001

10.5


In [ ]:
side_list = list()

with open ('data/exclude_ids.txt', 'r') as my_file:
  for line in my_file:
    side_list.append(line.rstrip())

p6 = beam.Pipeline()

class FilterUsingLength(beam.DoFn):
  def process(self, element, side_list, lower_bound, upper_bound=float('inf')):
    id = element.split(',')[0]
    name = element.split(',')[1]
    #id = id.decode('utf-8', 'ignore').encode("utf-8")
    element_list = element.split(',')
    if (lower_bound <= len(name) <= upper_bound) and id not in side_list:
      #print([element_list])
      return [element_list]

small_names = (p6
               | "Read from text file" >> beam.io.ReadFromText('data/dept_data.txt')
               | "ParDo with side inputs" >> beam.ParDo(FilterUsingLength(), side_list, 3, 10)
               #|beam.Map(print)
               | beam.Filter(lambda record: record[3] == "Accounts")
               | beam.Map(lambda record: (record[0] + " " + record[1], 1))
               | beam.CombinePerKey(sum)
               | "write results" >> beam.io.WriteToText('data/output1')
               )

p6.run()

In [ ]:
!head -n 10 data/output1-00000-of-00001

('503996WI Edouard', 31)
('957149WC Kyle', 31)
('241316NX Kumiko', 31)
('796656IE Gaston', 31)
('718737IX Ayumi', 30)


In [ ]:
class ProcessWords(beam.DoFn):
  def process(self, element, cutoff_length, marker):
    name = element.split(',')[1]

    if len(name) <= cutoff_length:
      return [beam.pvalue.TaggedOutput('Short_Names', name)]

    else:
      return [beam.pvalue.TaggedOutput('Long_Names', name)]

    if name.startswith(marker):
      return name

p = beam.Pipeline()

results = (p
           | beam.io.ReadFromText('data/dept_data.txt')
           | beam.ParDo(ProcessWords(), cutoff_length=4, marker='A').with_outputs(
              'Short_Names', 'Long_Names', main='Names_A'
           ))

short_collection = results.Short_Names
long_collection = results.Long_Names
startA_collection = results.Names_A

short_collection | 'Write 1' >> beam.io.WriteToText('short')

long_collection | 'Write 2' >> beam.io.WriteToText('long')

startA_collection | 'Write 3' >> beam.io.WriteToText('start_a')

p.run()

In [ ]:
!head -n 10 short-00000-of-00001

Itoe
Kyle
Kyle
Olga
Kirk
Itoe
Kyle
Kyle
Olga
Kirk


In [ ]:
!head -n 10 long-00000-of-00001

Marco
Rebekah
Edouard
Kumiko
Gaston
Beryl
Leslie
Mindy
Vicky
Richard


In [ ]:
!head -n 10 start_a-00000-of-00001